In [1]:
## Define Polynomial ring QQ[N], Partition algebras P_1(N) and P_2(N)
R.<N> = QQ[]
P1N = PartitionAlgebra(1,N)
P2N = PartitionAlgebra(2,N)

In [2]:
## We define the projector on the first slot in the vacillating tableau
def P_R1(R1):
    R.<N> = QQ[]
## Construct T2 \otimes \idn \otimes \idn defined in equation (3.42)
    Z1 = P2N(sum(P1N.jucys_murphy_element(i/2) for i in [1..2])+(N*(N-1)/2-N)*P1N.one())
## Define the set of irreps and corresponding normalized characters to take a product over in equation (3.52)
    IrrepsEigenvaluesDictionary = {Partition([]): R(1/2*(N-1)*N), Partition([1]): R((N-3)*N/2)}
    numerator = prod((Z1-ev2*P2N.one()) for (rep, ev2) in IrrepsEigenvaluesDictionary.items() if rep != R1)
    denom = prod(IrrepsEigenvaluesDictionary[R1]-ev2 for (rep,ev2) in IrrepsEigenvaluesDictionary.items() if rep != R1)
    return numerator, denom
## We define the projector on the second slot in the vacillating tableau
def P_R2(R2):
    R.<N> = QQ[]
## Construct T2 \otimes \idn \otimes \idn defined in equation (3.42)
    Z32 = P2N(sum(P2N.jucys_murphy_element(i/2) for i in [1..3])+(N*(N-1)/2-2*N+1)*P2N.one())
## Define the set of irreps and corresponding normalized characters to take a product over in equation (3.52)
    IrrepsEigenvaluesDictionary = {Partition([]): R(1/2*(N-2)*(N-1)), Partition([1]): R((N-4)*(N-1)/2)}
    numerator = prod((Z32-ev2*P2N.one()) for (rep, ev2) in IrrepsEigenvaluesDictionary.items() if rep != R2)
    denom = prod(IrrepsEigenvaluesDictionary[R2]-ev2 for (rep,ev2) in IrrepsEigenvaluesDictionary.items() if rep != R2)
    return numerator, denom
## We define the projector on the third slot in the vacillating tableau
def P_R3(R3):
    R.<N> = QQ[]
## Construct T2 \otimes \idn \otimes \idn defined in equation (3.42)
    Z2 = P2N(sum(P2N.jucys_murphy_element(i/2) for i in [1..4])+(N*(N-1)/2-2*N)*P2N.one())
## Define the set of irreps and corresponding normalized characters to take a product over in equation (3.52)
    IrrepsEigenvaluesDictionary = {Partition([]): R(1/2*(N-1)*N), Partition([1]): R((N-3)*N/2), Partition([2]): R(1/2*(N - 1)*(N - 4)), Partition([1,1]): R(1/2*(N - 5)*N)}
    numerator = prod((Z2-ev2*P2N.one()) for (rep, ev2) in IrrepsEigenvaluesDictionary.items() if rep != R3)
    denom = prod(IrrepsEigenvaluesDictionary[R3]-ev2 for (rep,ev2) in IrrepsEigenvaluesDictionary.items() if rep != R3)
    return numerator, denom

In [3]:
## The set of all vacillating tableaux at k=2
VacTabs = [ (Partition([]),Partition([]),Partition([])), \
            (Partition([]),Partition([]),Partition([1])), \
            (Partition([1]),Partition([1]),Partition([1])), \
            (Partition([1]),Partition([1]),Partition([2])), \
            (Partition([1]),Partition([1]),Partition([1,1])), \
            (Partition([1]),Partition([]),Partition([])), \
            (Partition([1]),Partition([]),Partition([1]))]

In [4]:
## Some initializations
g = {}
k = len(VacTabs)
## Now produce the diagram element corresponding to the connected 2-point function
## For this we need symbolic variables corresponding to coupling constants.
## We encode these in a dictionary
for v1 in range(k):
    for v2 in range(k):
        if v2 <= v1:
            g[(VacTabs[v1],VacTabs[v2])] = var('g_{0}{1}'.format(v1,v2))

In [ ]:
## We also collect the set of normalization constants computed in section 4.3.5
n = {}
n[0,0] = N
n[1,0] = N*sqrt(2/(N-1))
n[1,1] = N/(N-1)
n[2,2] = N
n[3,2] = N*sqrt(2/(N-1))
n[3,3] = N/(N-1)
n[4,2] = sqrt(2*(N-1)/(N-2))
n[4,3] = sqrt(2/(N-2))
n[4,4] = (N-1)
n[5,5] = 1
n[6,6] = 1

In [5]:
## We now generate all the matrix units and multiply by the corresponding coupling constant (which are symbolic)
## For this we need to use the partition algebra over a symbolic ring
N = var('N')
P2SR = PartitionAlgebra(2,N,SR)
## Some initializations
Q = 0
SRQ = 0
## We now iterate over all the pairs of vacillating tableaux
for v1 in range(k):
    LeftZip = list(zip(P_R3(VacTabs[v1][2]),P_R2(VacTabs[v1][1]),P_R1(VacTabs[v1][0])))
## LeftZip[0] is a list of the numerators of the three projectors for the first vacillating tableaux
    LeftProj = prod(LeftZip[0])
## We take their product to get the projector associated with the first vacillating tableaux
    for v2 in range(k):
        if v2 <= v1:
            RightZip = list(zip(P_R3(VacTabs[v2][2]),P_R2(VacTabs[v2][1]),P_R1(VacTabs[v2][0])))
## RightZip[0] is a list of the numerator of the three projectors for the second vacillating tableau
            RightProj = prod(RightZip[0])
## We take their product to get the projector associated with the second vacillating tableaux
            ProjMatrix = LeftProj.to_matrix(side='left')*RightProj.to_matrix(side='right')
## ProjMatrix is the matrix representing the simultaneous left action of LeftProj and right action of RightProj
            pivot = ProjMatrix.subs(N=10).pivots()
## Compute the pivot columns of ProjMatrix for N=10
            if len(pivot) > 0:
                Q = ProjMatrix.column(pivot[0])/prod(LeftZip[1])/prod(RightZip[1])
## As long as ProjMatrix has a pivot, extract the pivot column and divide by the numerators of the projectors
                SRQ += n[v1,v2]*g[(VacTabs[v1],VacTabs[v2])]*(P2SR.from_vector(Q)/2+ P2SR.from_vector(Q).dual()/2)
## We take the average of the pivot column and its transpose,
## weight it by the coupling constant associated with the pair of vacillating tableaux and add it to SRQ,
## SRQ will correspond to the connected two-point function (propagator) in our matrix model

In [6]:
## We define a vector space labeled by set partitions
## It has a "product" that combines set partitions by concatenation of lists: see product_on_basis()
## This product implements tensor products of diagrams
## It has some help functions, such as
## (1) a function that converts a partition algebra element into an element of this vector space: see from_partition_algebra()
## (2) a function that lets you change the label set of an element: see relabel_element()
## (3) an inner product function that computes the number of parts in the join of two set partitions (linearly extended for general elements): see inner_prod()
class SetPartitionNA(CombinatorialFreeModule):
    
    def __init__(self, **keywords):
        self._baseset = SetPartitions()
        CombinatorialFreeModule.__init__(self, SR, self._baseset,
            category=AlgebrasWithBasis(SR), **keywords)
        
    def product_on_basis(self, left, right):
        l = list(left)
        r = list(right)
        return self.monomial(SetPartition(l+r))
    
    def one_basis(self):
        return SetPartition([])
    
    def algebra_generators(self):
        return SetPartitions()
    
    def _repr_(self):
        return "Algebra of set partitions over %s with multiplication given by concatenation"%(SR)
    
    def inner_product_on_basis(self, left, right):
        if left.base_set() != right.base_set():
            return 0
        else:
            join = left.sup(right)
            return SR(N^len(join))
    
    def inner_prod(self, left, right):
        innerprod = 0
        for l in left:
            for r in right:
                innerprod += l[1]*r[1]*self.inner_product_on_basis(l[0],r[0])
        return innerprod
    
    def from_partition_algebra(self, d, baseset_left, baseset_right):
        B = self.basis()
        element = 0
        for (setpart, coeff) in d:
            setpartnew = []
            for part in setpart.set_partition():
                partnew = []
                for p in part:
                    if p > 0:
                        partnew += [baseset_left[p-1]]
                    elif p < 0:
                        partnew += [baseset_right[-p-1]]
                setpartnew += [partnew]
            element += coeff*B[SetPartition(setpartnew)]
        return element
    
    def relabel_element(self, elem, labelset):
        B = self.basis()
        element = 0
        for (setpart, coeff) in elem:
            setpartnew = []
            for part in setpart:
                partnew = []
                for p in part:
                    partnew += [labelset[p-1]]
                setpartnew += [partnew]
            element += coeff*B[SetPartition(setpartnew)]
        return element

In [7]:
## Initialize this vector space
A = SetPartitionNA()
B = A.basis()
## Note that, as it is currently defined, this is not an algebra.
## E.g. B[{{1}}]*B[{{1}}] = B[{{1},{1}}] is not a set partition.
## This can also be checked by running.
## TestSuite(A).run(verbose=True, skip='testpickling')
## However, the current form will suffice for our algorithm.

In [8]:
## Define Clebsches as elements of this vector space
gJ1, gJ2 = var('gJ1, gJ2')
C00 = 1/N*B[SetPartition([[1],[2]])]
CHH = 1/sqrt((N-1))*B[SetPartition([[1,2]])] - 1/N*1/sqrt((N-1))*B[SetPartition([[1],[2]])]
## The one point function is a linear combination of these
EXP_VAL = gJ1*C00+gJ2*CHH

In [9]:
## The connected two point function (propagator) is given by the partition algebra element SRQ computed earlier
PROPAGATOR = A.from_partition_algebra(SRQ, [1,2], [3,4])

In [ ]:
## Observables are specified by a set partiton. E.g.
observable_as_set_partition = B[SetPartition([[1,2]])]
## For a degree 1 observable
observable_as_set_partition = B[SetPartition([[1,2],[3,4]])]
## For a degree 2 observable
observable_as_set_partition = B[SetPartition([[1],[2],[3],[4],[5],[6]])]
## For a degree 3 observable

In [ ]:
##

In [22]:
## Degree 1 expvals are computed as
observable_as_set_partition = B[SetPartition([[1,2]])]
onepoint = EXP_VAL
A.inner_prod(observable_as_set_partition, onepoint)  

N*(gJ1/N - gJ2/(sqrt(N - 1)*N)) + N*gJ2/sqrt(N - 1)

In [18]:
## Degree 2 expvals are computed as
observable_as_set_partition = B[SetPartition([[1,2],[3,4]])]
twopoint = PROPAGATOR + EXP_VAL*A.relabel_element(EXP_VAL, [3,4])
A.inner_prod(observable_as_set_partition, twopoint)

((gJ1/N - gJ2/(sqrt(N - 1)*N))^2 - g_21/(N^2 - N) - g_22/(N^3 - 4*N^2 + 5*N - 2) + g_33/(N^2 - 3*N + 2) + g_62/(N^2 - N) + g_00/N^3 - g_11/N^3 - g_50/N^3 + g_55/N^3 + g_61/N^3 - g_66/N^3)*N^2 - (g_62*(1/(N^2 - N) + 1/(N - 1)) - 2*gJ2*(gJ1/N - gJ2/(sqrt(N - 1)*N))/sqrt(N - 1) - g_21/(N^2 - N) - 2*g_22/(N^3 - 4*N^2 + 5*N - 2) + 2*g_33/(N^2 - 3*N + 2) - g_50/N^2 + 2*g_55/N^2 + g_61/N^2 - 2*g_66/N^2)*N^2 + N^2*(gJ2^2/(N - 1) - g_22/(N^3 - 4*N^2 + 5*N - 2) + g_33/(N^2 - 3*N + 2) + g_55/N + g_62/(N - 1) - g_66/N) + N*(N*g_22/(N^3 - 4*N^2 + 5*N - 2) - N*g_33/(N - 2) - N*g_62/(N - 1) + g_66) + 1/2*N*(g_33 + g_44) + 1/2*N*(g_33 - g_44) + N*(g_62 - 2*g_22/(N^2 - 3*N + 2) + 2*g_33/(N - 2)) + 1/2*N*(2*g_21/(N^2 - N) + 2*g_22/(N^4 - 4*N^3 + 5*N^2 - 2*N) - g_33/(N - 2) + g_44/N - 2*g_62/(N^2 - N) + 2*g_11/N^2 - 2*g_61/N^2 + 2*g_66/N^2) - N*(g_21/(N - 1) + 2*g_22/(N^3 - 4*N^2 + 5*N - 2) - 2*g_33/(N - 2) - g_61/N - 2*g_62/(N - 1) + 2*g_66/N) + N*(g_21/N + 2*g_22/(N^3 - 3*N^2 + 2*N) - g_33/(N - 2) - g_

In [19]:
## Degree 3 expvals are computed as
observable_as_set_partition = B[SetPartition([[1],[2],[3],[4],[5],[6]])]
threepoint = PROPAGATOR*A.relabel_element(EXP_VAL, [5,6])
threepoint += A.relabel_element(PROPAGATOR, [1,2,5,6])*A.relabel_element(EXP_VAL, [3,4])
threepoint += A.relabel_element(PROPAGATOR, [3,4,5,6])*A.relabel_element(EXP_VAL, [1,2])
threepoint += A.relabel_element(EXP_VAL, [1,2])*A.relabel_element(EXP_VAL, [3,4])*A.relabel_element(EXP_VAL, [5,6])
A.inner_prod(observable_as_set_partition,threepoint)  

((gJ1/N - gJ2/(sqrt(N - 1)*N))^3 - 3*(gJ1/N - gJ2/(sqrt(N - 1)*N))*(g_21/(N^2 - N) + g_22/(N^3 - 4*N^2 + 5*N - 2) - g_33/(N^2 - 3*N + 2) - g_62/(N^2 - N) - g_00/N^3 + g_11/N^3 + g_50/N^3 - g_55/N^3 - g_61/N^3 + g_66/N^3))*N^6 + 3/2*N^5*(gJ1/N - gJ2/(sqrt(N - 1)*N))*(2*g_21/(N^2 - N) + 2*g_22/(N^4 - 4*N^3 + 5*N^2 - 2*N) - g_33/(N - 2) + g_44/N - 2*g_62/(N^2 - N) + 2*g_11/N^2 - 2*g_61/N^2 + 2*g_66/N^2) + 3*N^5*(gJ1/N - gJ2/(sqrt(N - 1)*N))*(g_21/N + 2*g_22/(N^3 - 3*N^2 + 2*N) - g_33/(N - 2) - g_44/N - g_62/N) + 3/2*N^5*(gJ1/N - gJ2/(sqrt(N - 1)*N))*(2*g_22/(N^2 - 2*N) - g_33/(N - 2) + g_44/N) + 3*(gJ2*(gJ1/N - gJ2/(sqrt(N - 1)*N))^2/sqrt(N - 1) - (g_62*(1/(N^2 - N) + 1/(N - 1)) - g_21/(N^2 - N) - 2*g_22/(N^3 - 4*N^2 + 5*N - 2) + 2*g_33/(N^2 - 3*N + 2) - g_50/N^2 + 2*g_55/N^2 + g_61/N^2 - 2*g_66/N^2)*(gJ1/N - gJ2/(sqrt(N - 1)*N)) - gJ2*(g_21/(N^2 - N) + g_22/(N^3 - 4*N^2 + 5*N - 2) - g_33/(N^2 - 3*N + 2) - g_62/(N^2 - N) - g_00/N^3 + g_11/N^3 + g_50/N^3 - g_55/N^3 - g_61/N^3 + g_66/N^3)/s

In [81]:
## For higher degree expvals it is useful to automate the Wick contractions
def kpoint(k):
## Fix a degree k
    pairs = [(2*i-1,2*i) for i in [1..k]]
## Generate the set of pairs [(1,2),....,(2k-1,2k)]
    kpoint=0
    for i in [0..floor(k/2)]:
        ## Iterate over all set partitions of the pairs with blocks of size 1 or 2
        for term in SetPartitions(pairs, sorted((k-2*i)*[1]+i*[2], reverse=True)):
            kpoint += prod(A.relabel_element(EXP_VAL, list(i for c in contraction for i in c)) if len(contraction)==1 else A.relabel_element(PROPAGATOR, list(i for c in contraction for i in c))  for contraction in term )
    return kpoint

In [88]:
## We can check that this gives the right answer for k=1,2,3
kpoint(1) == onepoint and kpoint(2) == twopoint and kpoint1(3) == threepoint

True

In [40]:
## Now we can compute degree 4 expectation values
observable_as_set_partition = B[SetPartition([[1],[2],[3],[4],[5],[6],[7],[8]])]
fourpoint=kpoint(4)
A.inner_prod(observable_as_set_partition, fourpoint)

((gJ1/N - gJ2/(sqrt(N - 1)*N))^4 - 6*(gJ1/N - gJ2/(sqrt(N - 1)*N))^2*(g_21/(N^2 - N) + g_22/(N^3 - 4*N^2 + 5*N - 2) - g_33/(N^2 - 3*N + 2) - g_62/(N^2 - N) - g_00/N^3 + g_11/N^3 + g_50/N^3 - g_55/N^3 - g_61/N^3 + g_66/N^3) + 3*(g_21/(N^2 - N) + g_22/(N^3 - 4*N^2 + 5*N - 2) - g_33/(N^2 - 3*N + 2) - g_62/(N^2 - N) - g_00/N^3 + g_11/N^3 + g_50/N^3 - g_55/N^3 - g_61/N^3 + g_66/N^3)^2)*N^8 + 2*(2*gJ2*(gJ1/N - gJ2/(sqrt(N - 1)*N))^3/sqrt(N - 1) - 3*(g_62*(1/(N^2 - N) + 1/(N - 1)) - g_21/(N^2 - N) - 2*g_22/(N^3 - 4*N^2 + 5*N - 2) + 2*g_33/(N^2 - 3*N + 2) - g_50/N^2 + 2*g_55/N^2 + g_61/N^2 - 2*g_66/N^2)*(gJ1/N - gJ2/(sqrt(N - 1)*N))^2 - 6*gJ2*(gJ1/N - gJ2/(sqrt(N - 1)*N))*(g_21/(N^2 - N) + g_22/(N^3 - 4*N^2 + 5*N - 2) - g_33/(N^2 - 3*N + 2) - g_62/(N^2 - N) - g_00/N^3 + g_11/N^3 + g_50/N^3 - g_55/N^3 - g_61/N^3 + g_66/N^3)/sqrt(N - 1) + 3*(g_62*(1/(N^2 - N) + 1/(N - 1)) - g_21/(N^2 - N) - 2*g_22/(N^3 - 4*N^2 + 5*N - 2) + 2*g_33/(N^2 - 3*N + 2) - g_50/N^2 + 2*g_55/N^2 + g_61/N^2 - 2*g_66/N^2)*(